![alt text](../img/conve.png)

In [1]:
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import sklearn
import pprint
import itertools
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
from data import WN18

In [2]:
class Config:
    n_epochs = 1000
    batch_size = 100
    embed_dim = 200

In [3]:
"""
e: entity
s: subject
p: predicate
o: object
"""
def glance_dict(d, n=5):
    return dict(itertools.islice(d.items(), n))


def read_triples(path):
    triples = []
    with open(path, 'rt') as f:
        for line in f.readlines():
            s, p, o = line.split()
            triples += [(s.strip(), p.strip(), o.strip())]
    return triples


def load_triple():
    WN18.download()
    triples_tr = read_triples('../data/WN18/wn18/train.txt')
    triples_va = read_triples('../data/WN18/wn18/valid.txt')
    triples_te = read_triples('../data/WN18/wn18/test.txt')
    triples_all = triples_tr + triples_va + triples_te
    return triples_all, triples_tr, triples_va, triples_te


def build_vocab(triples):
    params = {}
    e_set = {s for (s, p, o) in triples} | {o for (s, p, o) in triples}
    p_set = {p for (s, p, o) in triples}
    params['e_vocab_size'] = len(e_set)
    params['p_vocab_size'] = len(p_set)
    e2idx = {e: idx for idx, e in enumerate(sorted(e_set))}
    p2idx = {p: idx for idx, p in enumerate(sorted(p_set))}
    return e2idx, p2idx, params


def build_multi_label(triples_tr):
    sp2o = {}
    for (_s, _p, _o) in triples_tr:
        s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
        if (s,p) not in sp2o:
            sp2o[(s,p)] = [o]
        else:
            if o not in sp2o[(s,p)]:
                sp2o[(s,p)].append(o)
    return sp2o


def get_y(triples_tr, e2idx, p2idx, sp2o):
    y = []
    for (_s, _p, _o) in triples_tr:
        s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
        temp = np.zeros([len(e2idx)])
        temp[sp2o[(s,p)]] = 1.
        y.append(temp)
    y = np.asarray(y)
    return y


def next_train_batch(triples_tr, e2idx, p2idx, sp2o):
    for i in range(0, len(triples_tr), Config.batch_size):
        _triples_tr = triples_tr[i: i+Config.batch_size]
        x_s = np.asarray([e2idx[s] for (s, p, o) in _triples_tr], dtype=np.int32)
        x_p = np.asarray([p2idx[p] for (s, p, o) in _triples_tr], dtype=np.int32)
        y = get_y(_triples_tr, e2idx, p2idx, sp2o)
        yield (x_s, x_p, y)


def train_input_fn(triples_tr, e2idx, p2idx, s2p2o):
    dataset = tf.data.Dataset.from_generator(
        lambda: next_train_batch(sklearn.utils.shuffle(triples_tr),
                                 e2idx,
                                 p2idx,
                                 s2p2o),
        (tf.int32, tf.int32, tf.float32),
        (tf.TensorShape([None]),
         tf.TensorShape([None]),
         tf.TensorShape([None, len(e2idx)])))
    iterator = dataset.make_one_shot_iterator()
    x_s, x_p, y = iterator.get_next()
    return {'s': x_s, 'p': x_p}, y

In [4]:
def o_next_batch(eval_triples,
                 e2idx, 
                 p2idx):
    for (s, p, o) in tqdm(eval_triples, total=len(eval_triples), ncols=70):
        s_idx, p_idx = e2idx[s], p2idx[p]
        yield np.atleast_1d(s_idx), np.atleast_1d(p_idx)


def o_input_fn(eval_triples,
               e2idx, 
               p2idx):
    dataset = tf.data.Dataset.from_generator(
        lambda: o_next_batch(eval_triples,
                             e2idx, 
                             p2idx),
        (tf.int32, tf.int32),
        (tf.TensorShape([None,]),
         tf.TensorShape([None,])))
    iterator = dataset.make_one_shot_iterator()
    s, p = iterator.get_next()
    return {'s': s, 'p': p}


def evaluate_rank(model,
                  valid_triples,
                  test_triples,
                  all_triples,
                  e2idx,
                  p2idx):
    for eval_name, eval_triples in [('test', test_triples)]:
        _scores_o = list(model.predict(
            lambda: o_input_fn(eval_triples,
                               e2idx, 
                               p2idx)))
        ScoresO = np.reshape(_scores_o, [len(eval_triples), len(e2idx)])
        ranks_o, filtered_ranks_o = [], []
        for ((s, p, o), scores_o) in tqdm(zip(eval_triples, ScoresO),
                                          total=len(eval_triples),
                                          ncols=70):
            s_idx, p_idx, o_idx = e2idx[s], p2idx[p], e2idx[o]
            ranks_o += [1 + np.argsort(np.argsort(- scores_o))[o_idx]]
            filtered_scores_o = scores_o.copy()
            rm_idx_o = [e2idx[fo] for (fs, fp, fo) in all_triples if fs == s and fp == p and fo != o]
            filtered_scores_o[rm_idx_o] = - np.inf
            filtered_ranks_o += [1 + np.argsort(np.argsort(- filtered_scores_o))[o_idx]]
        for setting_name, setting_ranks in [('Raw', ranks_o), ('Filtered', filtered_ranks_o)]:
            mean_rank = np.mean(setting_ranks)
            print('[{}] {} Mean Rank: {}'.format(eval_name, setting_name, mean_rank))
            for k in [1, 3, 5, 10]:
                hits_at_k = np.mean(np.asarray(setting_ranks) <= k) * 100
                print('[{}] {} Hits@{}: {}'.format(eval_name, setting_name, k, hits_at_k))

In [ ]:
def forward(features, mode, params):
    batch_sz = tf.shape(features['s'])[0]
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    
    e_embed = tf.get_variable('e_embed',
                              [params['e_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    p_embed = tf.get_variable('p_embed',
                              [params['p_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    
    s = tf.nn.embedding_lookup(e_embed, features['s'])
    p = tf.nn.embedding_lookup(p_embed, features['p'])
    
    x = tf.concat([tf.reshape(s, [batch_sz, 10, 20, 1]),
                   tf.reshape(p, [batch_sz, 10, 20, 1]),], axis=1)
    
    x = tf.layers.dropout(x, 0.2, training=is_training)
    x = tf.layers.conv2d(x, 32, (3,3))
    x = tf.layers.batch_normalization(x, training=is_training)
    x = tf.nn.relu(x)
    
    x = tf.layers.flatten(x)
    
    x = tf.layers.dropout(x, 0.2, training=is_training)
    x = tf.layers.dense(x, Config.embed_dim)
    x = tf.layers.batch_normalization(x, training=is_training)
    x = tf.nn.relu(x)
    
    logits = tf.matmul(x, e_embed, transpose_b=True)
    return logits
    
    
def model_fn(features, labels, mode, params):
    logits = forward(features, mode, params)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        #tf.logging.info('\n'+pprint.pformat(tf.trainable_variables()))
        #tf.logging.info('params: %d'%count_train_params())
        
        loss_op = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,
                                                                        labels=labels))
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_op = tf.train.AdamOptimizer().minimize(
                loss_op, global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode = mode,
                                          loss = loss_op,
                                          train_op = train_op)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions = tf.sigmoid(logits))


def count_train_params():
    return np.sum([np.prod([d.value for d in v.get_shape()]) for v in tf.trainable_variables()])

In [ ]:
triples_all, triples_tr, triples_va, triples_te = load_triple()
e2idx, p2idx, params = build_vocab(triples_all)
sp2o = build_multi_label(triples_tr)

model = tf.estimator.Estimator(model_fn,
                               params = params)

for n_epoch in range(Config.n_epochs):
    tf.logging.info('Epoch %d'%(n_epoch+1))
    model.train(lambda: train_input_fn(triples_tr, e2idx, p2idx, sp2o))
    evaluate_rank(model,
                  triples_va,
                  triples_te,
                  triples_all,
                  e2idx,
                  p2idx,)

Files Already Downloaded
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp11ooa4sb', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1154636d8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Epoch 1
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorfl

100%|█████████████████████████████| 5000/5000 [01:47<00:00, 46.45it/s]


[test] Raw Mean Rank: 17477.6438
[test] Raw Hits@1: 1.6
[test] Raw Hits@3: 2.1399999999999997
[test] Raw Hits@5: 2.6599999999999997
[test] Raw Hits@10: 4.06
[test] Filtered Mean Rank: 17462.0306
[test] Filtered Hits@1: 1.6
[test] Filtered Hits@3: 2.26
[test] Filtered Hits@5: 2.68
[test] Filtered Hits@10: 4.24
INFO:tensorflow:Epoch 2
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp11ooa4sb/model.ckpt-1415
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1416 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp11ooa4sb/model.ckpt.
INFO:tensorflow:loss = 7500.4727, step = 1416
INFO:tensorflow:global_step/sec: 2.65686
INFO:tensorflow:loss = 5731.859, step = 1516 (37.640 sec)
INFO:tensorflow:global_step/sec: 2.77703
I

100%|█████████████████████████████| 5000/5000 [01:51<00:00, 44.65it/s]

[test] Raw Mean Rank: 15101.7374
[test] Raw Hits@1: 3.9600000000000004
[test] Raw Hits@3: 5.4
[test] Raw Hits@5: 5.56
[test] Raw Hits@10: 6.68
[test] Filtered Mean Rank: 15085.9588
[test] Filtered Hits@1: 4.18
[test] Filtered Hits@3: 5.64
[test] Filtered Hits@5: 5.82
[test] Filtered Hits@10: 7.12
INFO:tensorflow:Epoch 3


...

[test] Raw Mean Rank: 551.6016
[test] Raw Hits@1: 36.1
[test] Raw Hits@3: 49.68
[test] Raw Hits@5: 55.94
[test] Raw Hits@10: 64.22
[test] Filtered Mean Rank: 535.5806
[test] Filtered Hits@1: 46.08
[test] Filtered Hits@3: 58.599999999999994
[test] Filtered Hits@5: 63.88
[test] Filtered Hits@10: 70.88
INFO:tensorflow:Epoch 63
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp11ooa4sb/model.ckpt-87730
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 87731 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp11ooa4sb/model.ckpt.
INFO:tensorflow:loss = 8.846064, step = 87731
INFO:tensorflow:global_step/sec: 2.92262
INFO:tensorflow:loss = 11.227281, step = 87831 (34.217 sec)
INFO:tensorflow:global_step/sec: 2.94781
INFO:t

100%|█████████████████████████████| 5000/5000 [01:41<00:00, 49.33it/s]


[test] Raw Mean Rank: 553.5194
[test] Raw Hits@1: 35.76
[test] Raw Hits@3: 48.9
[test] Raw Hits@5: 55.800000000000004
[test] Raw Hits@10: 64.28
[test] Filtered Mean Rank: 537.4926
[test] Filtered Hits@1: 44.74
[test] Filtered Hits@3: 57.84
[test] Filtered Hits@5: 63.56
[test] Filtered Hits@10: 70.72
INFO:tensorflow:Epoch 64
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp11ooa4sb/model.ckpt-89145
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 89146 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp11ooa4sb/model.ckpt.
INFO:tensorflow:loss = 1.7664099, step = 89146
INFO:tensorflow:global_step/sec: 2.93087
INFO:tensorflow:loss = 0.7613076, step = 89246 (34.121 sec)
INFO:tensorflow:global_step/sec: 2.94463
INFO:

100%|█████████████████████████████| 5000/5000 [01:41<00:00, 49.35it/s]


[test] Raw Mean Rank: 551.8262
[test] Raw Hits@1: 35.74
[test] Raw Hits@3: 49.02
[test] Raw Hits@5: 55.14
[test] Raw Hits@10: 63.68000000000001
[test] Filtered Mean Rank: 535.7956
[test] Filtered Hits@1: 44.9
[test] Filtered Hits@3: 57.879999999999995
[test] Filtered Hits@5: 63.260000000000005
[test] Filtered Hits@10: 69.84
INFO:tensorflow:Epoch 65
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp11ooa4sb/model.ckpt-90560
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 90561 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp11ooa4sb/model.ckpt.
INFO:tensorflow:loss = 5.9423614, step = 90561
INFO:tensorflow:global_step/sec: 2.92267
INFO:tensorflow:loss = 2.1510885, step = 90661 (34.217 sec)
